# Edges and their frecuencies

### Import the libraries necessaries

In [1]:
import igraph as ig
import pandas as pd
import numpy as np
from multiprocessing import Pool
from time import time

### Import the data

The data import the different interactions among genes. The data has two columns: one whith 'INTERACTOR_A' and second whith 'INTERACTOR_B', and each row is a pair of genes who interact. So a same gen can appear several times on each column. 

It's importat you check and change the file paths at your PC. Don't forget it!!

In [2]:
data=pd.read_csv("/home/camilo/Documentos/Andres/Data_inicial.txt", delimiter='\t')

### Edge Adjusment 

Here it uses the pandas library to export the list with only interactors list in txt format. Then using Read_Ncol to create a graph just with edge list exported before; finally use simplify() to delete the duplicate edges and leave one of them.

It's importat you check and change the file paths at your PC. Don't forget it!!

In [3]:
Edges=data[['INTERACTOR_A','INTERACTOR_B']]
Edges.to_csv("/home/camilo/Documentos/Andres/Edges.txt",sep='\t',index=False,index_label=False,header=False)
G=ig.Graph.Read_Ncol("/home/camilo/Documentos/Andres/Edges.txt",names=True,weights=False,directed=False)
G.es['Frecuency']=G.count_multiple()
G.simplify(combine_edges='first')

## Comparing the gen ID's with a new list

The list which we want compare is imported.

It's importat you check and change the file paths at your PC. Don't forget it!!

In [4]:
global New_list_ID

In [5]:
New_list_ID=pd.read_csv('/home/camilo/Documentos/Andres/At_TFs_iTAK.txt', delimiter='\t',header=None)
New_list_ID=New_list_ID.rename(columns={0:'ID',1:'Family',2:'Type'})

Here it declarates the list of edges with the name gen

In [6]:
Vs=pd.DataFrame(ig.VertexSeq(G)['name']).to_dict()
Edges=pd.DataFrame(G.get_edgelist())
Edges.replace(Vs[0],inplace=True)
Edges=Edges.rename(columns={0:'INTERACTOR_A',1:'INTERACTOR_B'})

This couples_TF function states if a edges is found at the new list, or if any node of edge is on it.

In [7]:
def Couples_TF(edge):
    place_1=np.where(edge[0]==New_list_ID.to_numpy()[:,0])
    place_2=np.where(edge[1]==New_list_ID.to_numpy()[:,0])
    if(np.array(place_1[0]).any()!=0 and np.array(place_2[0]).any()!=0):
        bool=True
        x=True
        y=True
        return [bool,x,y]
    elif(np.array(place_1[0]).any()!=0 or np.array(place_2[0]).any()!=0):
        bool=False
        x=np.array(place_1[0]).any()!=0
        y=np.array(place_2[0]).any()!=0
        return [bool,x,y]
    else:
        x=False
        y=False
        bool=False
        return [bool,x,y]

Parallelization process

In [8]:
pool=Pool() #Here define the number of cores, you can change depending of your PC
start_time=time()
result=pool.map(Couples_TF,Edges.to_numpy())
end_time=time()-start_time
pool.close()
pool.join()
print('process time:' ,end_time,'s')

process time: 0.03089761734008789 s


In [9]:
Edges['TF_both']=np.array(result)[:,0]
Edges['TF_A']=np.array(result)[:,1]
Edges['TF_B']=np.array(result)[:,2]
Edges

,INTERACTOR_A,INTERACTOR_B,TF_both,TF_A,TF_B
0,AT4G00020,AT5G20850,False,False,False
1,AT4G00020,AT3G22880,False,False,False
2,AT5G61380,AT2G43010,True,True,True
3,AT5G61380,AT3G59060,True,True,True
4,AT5G61380,AT2G46790,True,True,True
...,...,...,...,...,...
103,AT5G11260,AT1G56650,True,True,True
104,AT4G02570,AT5G25350,False,False,False
105,AT4G02570,AT2G25490,False,False,False
106,AT4G02570,AT5G20570,False,False,False
